This notebook is written to experiment different type of RAG pipeline techniques using LLAMA_INDEX.
The performance of each RAG technique is evaluated by Trulens.
The evaluation is based on 1.Answer Relevance, 2. Context Relevance and 3.Groundedness.

Below are RAG Techniques.
1. Basic RAG pipeline 
2. Sentence Window retrieval
3. Auto-merging retrieval

There is a utility module i.e utils.py which is called in this notebook to envoke different RAG technique.

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
# from llama_index.core import SimpleDirectoryReader

# documents = SimpleDirectoryReader(
#     input_files=["data/book.pdf"]
    
# ).load_data()

In [2]:
import os
from glob import glob
from llama_index.core import SimpleDirectoryReader

# Get all PDF files in the "data" folder
pdf_files = glob("data/*.pdf")
print(pdf_files)
# Create SimpleDirectoryReader instance with the list of PDF files
documents = SimpleDirectoryReader(input_files=pdf_files).load_data()


['data/book.pdf', 'data/tenancyguideen.pdf', 'data/tr01-survey-manual-sign_2022-04-26.pdf', 'data/rental-increase-decree-43-in-dubai.pdf']


In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

180 

<class 'llama_index.core.schema.Document'>
Doc ID: 72e74ea4-9be9-4d40-8cae-80d16c80edb0
Text:


## Basic RAG pipeline

In [4]:
from llama_index.core import Document, VectorStoreIndex

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

/var/folders/sx/55mj_5d53ps8zvqrbm12z_xh0000gp/T/ipykernel_41877/2769001838.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "How much is the fine for The fine for a person who is not registered on the roll?"
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The fine for a person who is not registered on the roll is double the prescribed fee.


In [8]:
response = query_engine.query(
    "How much is the fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution?"
)
print(str(response))

The fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution is double the prescribed Fee.


## Evaluation setup using TruLens

In [9]:
eval_questions = []
with open('evaluation_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

How much is the fine for the person who is not registered on the roll?
How much is the fine for obstructing the work of RERA employees?
How much is the fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution?
What is  Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai all about?
What is Article(3) Scope of Application for Law No.(6)
What is Article(4) for Jointly Owned Real Property Register for Law No.(6)
What is Article (5) for Ownership Rules
What is Jointly Owned Real Property Title Deeds Article (6)
What is Components of Common Parts Article (7)  of Law No. (6)
Which law talks about rental increase decree for properties in Emirates Dubai?
What is Article (1) for Percentage of increase for rental increase decree for properties in Emirates Dubai?
What does Article (2) for Scope of Application says about?
What does the Article (3) realtes to?
What is Article (3) for rental increase decree?
What is tena

In [10]:
# You can try your own question:
new_question = "How to find the best tenancy option in Dubai?"
eval_questions.append(new_question)

In [11]:
print(eval_questions)

['How much is the fine for the person who is not registered on the roll?', 'How much is the fine for obstructing the work of RERA employees?', 'How much is the fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution?', 'What is  Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai\xa0all about?', 'What is Article(3) Scope of Application for Law No.(6)', 'What is Article(4) for Jointly Owned Real Property Register for Law No.(6)', 'What is Article (5)\xa0for Ownership Rules', 'What is Jointly Owned Real Property Title Deeds Article (6)', 'What is Components of Common Parts Article (7)  of Law No. (6)', 'Which law talks about rental increase decree for properties in Emirates Dubai?', 'What is Article (1) for Percentage of increase for rental increase decree for properties in Emirates Dubai?', 'What does Article (2) for Scope of Application says about?', 'What does the Article (3) realtes to?', 'What is Artic

In [12]:
response = query_engine.query(
    "What is  Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai all about?"
)
print(str(response))

Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai is about regulations and rules related to jointly owned real property in Dubai, including definitions of terms, the scope of application, the establishment of a register for jointly owned real property, ownership rules, title deeds for jointly owned real property, and obligations of developers in relation to filing necessary documents with the Dubai Land Department.


In [13]:
response = query_engine.query(
    "What is Article(3) Scope of Application for Law No.(6)"
)
print(str(response))

The provisions of Law No. (6) apply to all Master Projects and Jointly Owned Real Property in the Emirate, including those in Special Development Zones and in free zones.


In [14]:
response = query_engine.query(
    "What is Article(4) for Jointly Owned Real Property Register for Law No.(6)"
)
print(str(response))

Article (4) for Jointly Owned Real Property Register under Law No. (6) includes maintaining a special register of Jointly Owned Real Property by the Dubai Land Department (DLD). This register contains details of land plots owned by Developers, Units intended for individual ownership, names of Owners, details of Owners Committees, Building Management Regulations, Plans, details of Management Entities, contracts for management of Jointly Owned Real Property, total area of Common Parts and Designated Common Parts, details of Developer-owned Areas, and provisions for issuing certificates, deeds, or other related documents based on the register information upon request.


In [15]:
response = query_engine.query(
    "What is Article (5) for Ownership Rules?"
)
print(str(response))

Article (5) states that original documents and court judgments related to the registration of Real Property must be kept by the DLD and cannot be removed from its premises. Interested parties, judicial authorities, or appointed experts and committees may access these documents and obtain certified copies as per the provisions of the law.


In [16]:
response = query_engine.query(
    "What is Jointly Owned Real Property Title Deeds Article (6)?"
)
print(str(response))

The Plans, Master Community Declaration, Statute, and Building Management Regulation constitute part of the title deed of Jointly Owned Real Property. The Dubai Land Department (DLD) maintains an original copy of each of these documents.


In [17]:
response = query_engine.query(
    "What is Components of Common Parts Article (7)  of Law No. (6)?"
)
print(str(response))

The Components of Common Parts Article (7) of Law No. (6) include various elements such as structural components, thresholds, entrances, utilities systems, elevators, water mains, equipment for Utility Services, and any other necessary parts for the existence and operation of the Jointly Owned Real Property.


In [18]:
response = query_engine.query(
    "Which law talks about rental increase decree for properties in Emirates Dubai?"
)
print(str(response))

Decree No (43) for 2013 regarding the Determination of the Increase in Rent for Properties in the Emirate of Dubai.


In [19]:
response = query_engine.query(
    "What is Article (1) for Percentage of increase for rental increase decree for properties in Emirates Dubai?"
)
print(str(response))

Article (1) states that when renewing real property lease contracts in the Emirate of Dubai, the maximum percentage of rent increase for real property will be as follows: a. no rent increase, where the rent of the real property unit is up to ten percent (10%) less than the average rental value of similar units; b. five percent (5%) of the rent of the real property unit, where the rent is eleven percent (11%) to twenty percent (20%) less than the average rental value of similar units; c. ten percent (10%) of the rent of the real property unit, where the rent is twenty-one percent (21%) to thirty percent (30%) less than the average rental value of similar units; d. fifteen percent (15%) of the rent of the real property unit, where the rent is thirty-one percent (31%) to forty percent (40%) less than the average rental value of similar units; or e. twenty percent (20%) of the rent of the real property unit, where the rent is more than forty percent (40%) less than the average rental value

In [20]:
response = query_engine.query(
    "What does Article (2) for Scope of Application says about?"
)
print(str(response))

Article (2) for Scope of Application outlines the specific regulations and provisions that apply to the registration, obligations, and responsibilities of real property brokers within the real estate sector.


In [21]:
response = query_engine.query(
    "What does the Article (3) realtes to?"
)
print(str(response))

Article (3) relates to the submission of an application for registration of a legal disposition of a Real Property Unit after the expiration of a specified time limit, the actions required by the DLD in such a situation, and the fine imposed on the Developer for the delay.


In [22]:
response = query_engine.query(
    "What is Article (3) for rental increase decree?"
)
print(str(response))

For the rental increase decree, Article (3) specifies that the average rental value of similar units will be determined according to the "Rent Index of the Emirate of Dubai" approved by the Real Estate Regulatory Agency.


In [23]:
response = query_engine.query(
    "What is tenancy all about?"
)
print(str(response))

Tenancy involves the contractual relationship between a Landlord and a Tenant regarding the use of Real Property. It is regulated by a detailed Tenancy Contract that specifies the description of the leased property, the purpose of tenancy, the term of the contract, the Rent and payment method, and the owner's name if the Landlord is not the owner. The Rent must be specified in the contract, and if not, it will be determined based on similar properties. The parties can agree to renew the contract, adjust Rent, or amend terms before expiry. The Landlord must hand over the property in good condition, and eviction can occur in specific cases outlined by the law.


In [24]:
response = query_engine.query(
    "What is Ejari Mechanism in Dubai?"
)
print(str(response))

EJARI Mechanism in Dubai involves the mandatory registration of tenancy contracts through the EJARI online program. It aims to connect all parties involved in property lease and management, including licensed companies, property owners, and landlords, to register and issue contracts. Additionally, EJARI facilitates the registration of management contracts between property owners and licensed real estate companies to define responsibilities and obligations related to property management, tenant services, lease contracts, legal procedures, rent collection, maintenance, and other related tasks.


In [25]:
response = query_engine.query(
    "What are the documents required for registration in Ejari?"
)
print(str(response))

The documents required for registration in EJARI are as follows:
- For Owner Manager (Individual): Passport copy and copy of the title deed of any owned property.
- For Owner Manager (Company): Copy of the trade license, copy of the title deed of any property owned by the company, copy of the passport of the official representative of the company, letter from the company identifying the qualified users, and copies of the passports of the employees nominated to use the program.


In [26]:
response = query_engine.query(
    "How to Register Lease Contracts with RERA?"
)
print(str(response))

To register lease contracts with RERA, the applicant needs to fill the online form accurately, provide all required documents such as the original lease contract, DEWA invoice, copy of the land map or title deed, copy of the tenant's passport or trade license if a company, and ensure that all information matches with the lease contract. The lease contract should not involve sublease agreements, must adhere to property construction standards, and the purpose of lease and type of use must comply with regulations in the Emirate of Dubai. Additionally, attention should be paid to the contract duration and property type to ensure it aligns with the intended use.


In [27]:

response = query_engine.query(
    "What is Tenancy Regulating Legislations?"
)
print(str(response))

Tenancy Regulating Legislations refer to the set of laws and regulations that govern the relationship between landlords and tenants in a specific jurisdiction, in this case, the Emirate of Dubai. These legislations outline the rights and responsibilities of both parties, the procedures for tenancy contracts, rent determination, property conditions, eviction reasons, and dispute resolution mechanisms.


In [28]:

response = query_engine.query(
    "How many articles are there in Tenancy regulating legislations?"
)
print(str(response))

There are 7 articles in the Tenancy regulating legislations.


In [29]:

response = query_engine.query(
    "How to regulate the relationship between Landlords and Tenants in the Emirates of Dubai?"
)
print(str(response))

Regulate the relationship between Landlords and Tenants in the Emirates of Dubai by ensuring that a written Lease Contract is signed by both parties, specifying details such as the description of the leased Real Property, the purpose of the Lease Contract, the term of the lease, the Rent amount, and the Rent payment method. Additionally, all Lease Contracts must be registered with the Real Estate Regulatory Agency (RERA), and any amendments to the Lease Contracts should also be registered. It is important to adhere to the regulations set forth in the Law to govern the relationship effectively.


In [30]:
response = query_engine.query(
    "What is rent Index?"
)
print(str(response))

The Rent Index is a tool created by RERA in the Emirate of Dubai to determine the rent values of properties of various uses in different regions. It serves as a reference point in case of disputes between landlords and tenants regarding rental rates and increases. The Index includes information on types of properties, rental units, minimum and maximum rent values, and average rent payments. It is divided into regions such as Deira, Dubai, and freehold areas, and covers property uses like residential, commercial, and industrial. Additionally, there are rent calculators available for commercial complexes and malls based on their classifications.


In [31]:
response = query_engine.query(
    "What is Rent Increase Calculator?"
)
print(str(response))

Rent Increase Calculator is a tool used to determine the rates of increase in rents and identify the rent values in various areas. It allows users to input required data to inquire about the increase rate and the average rent for the area and the unit.


In [32]:
response = query_engine.query(
    "Can you summarize this whole book?"
)
print(str(response))

This book outlines the regulations and procedures related to tenancy contracts, landlords, tenants, and real property in the Emirate of Dubai. It covers aspects such as the registration of tenancy contracts, determination of rent, renewal of contracts, amending contract terms, dispute settlement procedures, and penalties for real property brokers who violate regulations. The book also includes information on the responsibilities of landlords and tenants, the process of handing over real property, and the requirements for brokers operating in Dubai.


In [33]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [34]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [35]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [36]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [37]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_fd01f13d48e8d30ae0e01bd7e3ef85cc,"""How much is the fine for the person who is no...","""The fine for a person who is not registered o...",-,"{""record_id"": ""record_hash_fd01f13d48e8d30ae0e...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T14:35:40.473114"", ""...",2024-02-25T14:35:43.436465,0.9,0.0,0.0,[{'args': {'prompt': 'How much is the fine for...,[{'args': {'source': '000.00 30% of the marke...,[{'args': {'prompt': 'How much is the fine for...,2,2156,0.003243
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_9663e79fb73b5433289a25c47fee5bd4,"""How much is the fine for obstructing the work...","""The fine for obstructing the work of RERA emp...",-,"{""record_id"": ""record_hash_9663e79fb73b5433289...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T14:35:43.808532"", ""...",2024-02-25T14:35:46.206843,1.0,0.0,0.0,[{'args': {'prompt': 'How much is the fine for...,[{'args': {'source': 'f. Where the cash reserv...,[{'args': {'prompt': 'How much is the fine for...,2,2146,0.003230
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_613ba0373ef43d5d08c8754f0bddcb8c,"""How much is the fine for Failure to comply wi...","""The fine for Failure to comply with the oblig...",-,"{""record_id"": ""record_hash_613ba0373ef43d5d08c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T14:35:46.523445"", ""...",2024-02-25T14:35:48.156325,1.0,0.0,0.1,[{'args': {'prompt': 'How much is the fine for...,[{'args': {'source': 'Fees for dispositions ot...,[{'args': {'prompt': 'How much is the fine for...,1,2017,0.003038
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_22f2937a3e8091b312cf04c038f39f97,"""What is Law No. (6) of 2019 Concerning Owner...","""Law No. (6) of 2019 Concerning Ownership of J...",-,"{""record_id"": ""record_hash_22f2937a3e8091b312c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T14:35:48.497956"", ""...",2024-02-25T14:36:52.040698,1.0,1.0,1.0,[{'args': {'prompt': 'What is Law No. (6) of ...,[{'args': {'source': '(26) of 2013 Concerning ...,[{'args': {'prompt': 'What is Law No. (6) of ...,63,1913,0.002922
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_e4b2ad14bb35027475c284ed7fd8b5de,"""What is Article(3) Scope of Application for L...","""The provisions of Law No. (6) apply to all Ma...",-,"{""record_id"": ""record_hash_e4b2ad14bb35027475c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-02-25T14:36:52.352592"", ""...",2024-02-25T14:36:56.035737,1.0,0.5,0.6,[{'args': {'prompt': 'What is Article(3) Scope...,[{'args': {'source': 'Article (7) The Division...,[{'args': {'prompt': 'What is Article(3) Scope...,3,1872,0.002826


In [38]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.70.101:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [39]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [40]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [41]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [42]:
window_response = sentence_window_engine.query(
    "How much is the fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution?"
)
print(str(window_response))

The fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution is not explicitly mentioned in the provided context information.


In [43]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/utils/python.py", line 374, in _future_target_wrapper
    return func(*args, **kwargs)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/app.py", line 559, in _manage_pending_feedback_results
    record.wait_for_feedback_results()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/schema.py", line 277, in wait_for_feedback_results
    feedback_result = future_result.result()
  File "

In [ ]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

How much is the fine for the person who is not registered on the roll?
The fine for a person who is not registered on the roll is 20,000 Dirhams.
How much is the fine for obstructing the work of RERA employees?
The fine for obstructing the work of RERA employees is AED 50,000.
How much is the fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution?
The fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution is not explicitly mentioned in the provided context information.
What is  Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai all about?
Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai is a legislation that addresses the ownership and management of jointly owned real property in Dubai.
What is Article(3) Scope of Application for Law No.(6)
The provisions of Law No. (6) apply to the Real Property existing in the Emirat

Run of run in <Thread(TP.submit with debug timeout_3, started 123145697374208)> timed out after 600.0 second(s).
None
exception calling callback for <Future at 0x7fc8dc976460 state=finished raised TimeoutError>
Traceback (most recent call last):
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/_base.py", line 330, in _invoke_callbacks
    callback(self)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/app.py", line 1168, in _add_future_feedback
    res = future_result.result()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/_base.py", line 439, in result
    return self.__get_result()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/_base.py", line 391, in __get_result
    raise self._exception
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/Users/tarique/anaconda3

In [45]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.722222,0.982609,0.7375,10.208333,0.00129


In [46]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.70.101:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [47]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [48]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [49]:
auto_merging_response = automerging_query_engine.query(
    "How much is the fine for obstructing the work of RERA employees?"
)
print(str(auto_merging_response))

The fine for obstructing the work of RERA employees is not specified in the provided context information.


In [50]:
auto_merging_response = automerging_query_engine.query(
    "How much is the fine for the person who is not registered on the roll?"
)
print(str(auto_merging_response))

The fine for a person who is not registered on the roll is described as a fee in Dirhams.


In [51]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [52]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

How much is the fine for the person who is not registered on the roll?
The fine for a person who is not registered on the roll is specified as a fee in Dirhams.
How much is the fine for obstructing the work of RERA employees?
The fine for obstructing the work of RERA employees is 50,000 Dirhams.
How much is the fine for Failure to comply with the obligations stipulated in Article (16) of this Resolution?
The fine for failure to comply with the obligations stipulated in Article (16) of this Resolution is not less than one hundred thousand Dirhams (AED 100,000.00).
What is  Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai all about?
Law No. (6) of 2019 Concerning Ownership of Jointly Owned Real Property in the Emirate of Dubai is a legal document that defines regulations related to jointly owned real property in Dubai. It outlines the rights and responsibilities of individuals or entities who own property jointly in the Emirate of Dubai.
> M

Run of run in <Thread(TP.submit with debug timeout_19, started 123146234638336)> timed out after 600.0 second(s).
None
exception calling callback for <Future at 0x7fc8a9d56cd0 state=finished raised TimeoutError>
Traceback (most recent call last):
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/_base.py", line 330, in _invoke_callbacks
    callback(self)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/app.py", line 1168, in _add_future_feedback
    res = future_result.result()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/_base.py", line 439, in result
    return self.__get_result()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/_base.py", line 391, in __get_result
    raise self._exception
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/Users/tarique/anaconda

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/utils/python.py", line 374, in _future_target_wrapper
    return func(*args, **kwargs)
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/app.py", line 559, in _manage_pending_feedback_results
    record.wait_for_feedback_results()
  File "/Users/tarique/anaconda3/envs/rag/lib/python3.9/site-packages/trulens_eval/schema.py", line 277, in wait_for_feedback_results
    feedback_result = future_result.result()
  File "

In [53]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.784028,0.979167,0.70625,8.375,0.000715


In [54]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.70.101:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>